In [13]:
import pandas as pd
import geopandas as gpd
import descartes
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

%matplotlib inline

Import the necessary resources before starting

In [50]:
neighborhood = gpd.read_file("Neighborhoods_.shp") #Read in file

locations = pd.read_csv("https://data.wprdc.org/datastore/dump/9ed126cc-3c06-496e-bd08-b7b6b14b4109")
query_mask = locations['status'] == "Active"
latitude_mask= locations['latitude'].notna()
longitude_mask = locations['longitude'].notna()


locations_active = locations[query_mask & latitude_mask & longitude_mask]
locations_active.head(10)



geometry = [Point(xy) for xy in zip(locations_active['latitude'], locations_active['longitude'])]

locations_df = gpd.GeoDataFrame(locations_active, crs = 'epsg:4326', geometry = geometry)


fig, ax = plt.subplots(figsize=(15,15))

neighborhood.plot(ax=ax, alpha=0.4, color='gray')

locations_df[locations_df['location_type'] == "On street"].plot(ax=ax, markersize=20, color='blue', marker="^", label="On Street")